# Aprendizaje automático I

## Proceso de ETL (Extracción, Transformación y Carga)

In [2]:
# Importar la función merge_data desde load.py
from py.load import merge_data
import pandas as pd

# Ejecutar la función
merge_data('data/original/bookings_train.csv', 'data/original/hotels.csv', 'data/process/data.csv')
# Cargar el DataFrame desde el archivo CSV
df = pd.read_csv('data/process/data.csv')
# Veo las dimensiones del DataFrame
df

Datos cargados y guardados en: data/data.csv


,board,country_x,market_segment,distribution_channel,room_type,required_car_parking_spaces,special_requests,stay_nights,rate,total_guests,...,booking_date,reservation_status_date,reservation_status,hotel_type,country_y,parking,total_rooms,restaurant,pool_and_spa,avg_review
0,BB,SPA,Corporate,Corporate,A,NaN,1.0,1.0,65.0,3.0,...,2016-12-09,2016-12-13,Check-Out,City Hotel,SPA,False,126,True,False,4.75
1,NaN,SPA,Corporate,Corporate,A,NaN,1.0,2.0,130.0,3.0,...,2016-10-17,2016-10-19,Check-Out,City Hotel,SPA,False,126,True,False,4.75
2,BB,SPA,Corporate,Corporate,A,NaN,1.0,1.0,65.0,3.0,...,2016-07-22,2016-07-26,Check-Out,City Hotel,SPA,False,126,True,False,4.75
3,BB,SPA,Corporate,Corporate,A,NaN,1.0,2.0,NaN,3.0,...,2017-01-07,2017-01-12,Check-Out,City Hotel,SPA,False,126,True,False,4.75
4,BB,SPA,Corporate,Corporate,A,0.0,1.0,1.0,65.0,3.0,...,2016-11-21,2016-11-22,Check-Out,City Hotel,SPA,False,126,True,False,4.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50736,BB,POR,Direct,Direct,A,NaN,2.0,4.0,476.0,4.0,...,2016-05-10,2016-08-28,Check-Out,Resort Hotel,SPA,False,45,True,False,4.54
50737,HB,ITA,Online TA,TA/TO,E,0.0,2.0,10.0,2062.0,4.0,...,2016-05-01,2016-08-26,Check-Out,Resort Hotel,SPA,False,45,True,False,4.54
50738,BB,FRA,Online TA,TA/TO,E,0.0,2.0,4.0,836.0,4.0,...,2016-07-13,2016-08-26,Check-Out,Resort Hotel,SPA,False,45,True,False,4.54
50739,BB,IRL,Offline TA/TO,TA/TO,A,NaN,2.0,7.0,719.6,4.0,...,2016-05-09,2016-08-27,Check-Out,Resort Hotel,SPA,False,45,True,False,4.54


In [14]:
from py.split import split_train_validation

# Ejecutar la función para dividir los datos
df_train, df_validation = split_train_validation(df, '2017-06-30', 'data/process/data_validation.csv')

# Veo las dimensiones de los DataFrames
df_train.shape, df_validation.shape

El DataFrame df_validation se ha guardado en 'data/data_validation.csv'.
Se han creado los DataFrames df_train y df_validation.


((46602, 22), (1407, 22))

In [15]:
df_train

,board,country_x,market_segment,distribution_channel,room_type,required_car_parking_spaces,special_requests,stay_nights,rate,total_guests,...,booking_date,reservation_status_date,reservation_status,hotel_type,country_y,parking,total_rooms,restaurant,pool_and_spa,avg_review
0,BB,SPA,Corporate,Corporate,A,NaN,1.0,1.0,65.0,3.0,...,2016-12-09,2016-12-13,Check-Out,City Hotel,SPA,False,126,True,False,4.75
1,NaN,SPA,Corporate,Corporate,A,NaN,1.0,2.0,130.0,3.0,...,2016-10-17,2016-10-19,Check-Out,City Hotel,SPA,False,126,True,False,4.75
2,BB,SPA,Corporate,Corporate,A,NaN,1.0,1.0,65.0,3.0,...,2016-07-22,2016-07-26,Check-Out,City Hotel,SPA,False,126,True,False,4.75
3,BB,SPA,Corporate,Corporate,A,NaN,1.0,2.0,NaN,3.0,...,2017-01-07,2017-01-12,Check-Out,City Hotel,SPA,False,126,True,False,4.75
4,BB,SPA,Corporate,Corporate,A,0.0,1.0,1.0,65.0,3.0,...,2016-11-21,2016-11-22,Check-Out,City Hotel,SPA,False,126,True,False,4.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50736,BB,POR,Direct,Direct,A,NaN,2.0,4.0,476.0,4.0,...,2016-05-10,2016-08-28,Check-Out,Resort Hotel,SPA,False,45,True,False,4.54
50737,HB,ITA,Online TA,TA/TO,E,0.0,2.0,10.0,2062.0,4.0,...,2016-05-01,2016-08-26,Check-Out,Resort Hotel,SPA,False,45,True,False,4.54
50738,BB,FRA,Online TA,TA/TO,E,0.0,2.0,4.0,836.0,4.0,...,2016-07-13,2016-08-26,Check-Out,Resort Hotel,SPA,False,45,True,False,4.54
50739,BB,IRL,Offline TA/TO,TA/TO,A,NaN,2.0,7.0,719.6,4.0,...,2016-05-09,2016-08-27,Check-Out,Resort Hotel,SPA,False,45,True,False,4.54


In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split

def preprocess_data(df_train, test_size=0.3, random_state=42):
    """
    Realiza el preprocesamiento inicial de los datos y los divide en train y test.
    """

    # Eliminar hotel_id y country_x y days_diff
    df_train.drop(columns=['hotel_id', 'country_x'], errors='ignore', inplace=True)

    # Renombrar la columna 'country_y' a 'country'
    if 'country_y' in df_train.columns:
        df_train.rename(columns={'country_y': 'country'}, inplace=True)

    # Transformaciones de fechas y creación de variable target
    df_train['booking_date'] = pd.to_datetime(df_train['booking_date'])
    df_train['arrival_date'] = pd.to_datetime(df_train['arrival_date'])
    df_train['reservation_status_date'] = pd.to_datetime(df_train['reservation_status_date'])

    df_train["days_diff"] = (pd.to_datetime(df_train["arrival_date"]) - pd.to_datetime(df_train["reservation_status_date"])).dt.days
    df_train["cancelled_last_30_days"] = ((df_train["reservation_status"] == "Canceled") & (df_train["days_diff"] <= 30)).astype(int)
    df_train.drop(columns=['reservation_status', 'days_diff'], inplace=True)

    # Cálculo de la anticipación de la reserva
    df_train['advance_reservation_days'] = (pd.to_datetime(df_train['arrival_date']) - pd.to_datetime(df_train['booking_date'])).dt.days
    # Eliminar columnas de fecha
    date_cols = ['booking_date', 'arrival_date', 'reservation_status_date']
    df_train.drop(columns=date_cols, inplace=True)

    # De boleano a entero
    df_train['parking'] = df_train['parking'].astype(int)
    df_train['restaurant'] = df_train['restaurant'].astype(int)
    df_train['pool_and_spa'] = df_train['pool_and_spa'].astype(int)

    # Manejo de outliers en 'rate' y 'total_guests' usando el rango intercuartil (IQR)
    q1_rate = df_train["rate"].quantile(0.25)
    q3_rate = df_train["rate"].quantile(0.75)
    iqr_rate = q3_rate - q1_rate
    lower_bound_rate = q1_rate - 1.5 * iqr_rate
    upper_bound_rate = q3_rate + 1.5 * iqr_rate
    df_train = df_train[(df_train["rate"] >= lower_bound_rate) & (df_train["rate"] <= upper_bound_rate)]

    q1_guests = df_train["total_guests"].quantile(0.25)
    q3_guests = df_train["total_guests"].quantile(0.75)
    iqr_guests = q3_guests - q1_guests
    lower_bound_guests = q1_guests - 1.5 * iqr_guests
    upper_bound_guests = q3_guests + 1.5 * iqr_guests
    df_train = df_train[(df_train["total_guests"] >= lower_bound_guests) & (df_train["total_guests"] <= upper_bound_guests)]

    # Imputar valores nulos específicos
    df_train['special_requests'].fillna(0, inplace=True)
    df_train['required_car_parking_spaces'].fillna(0, inplace=True)

    # Separar la variable objetivo y las predictoras
    X = df_train.drop('cancelled_last_30_days', axis=1)
    y = df_train['cancelled_last_30_days']

    # Dividir en train y test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    return X_train, X_test, y_train, y_test

# 2. Preprocesamiento inicial (división train/test, manejo de nulos/outliers, etc.)
X_train, X_test, y_train, y_test = preprocess_data(df_train)

In [24]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((25807, 17), (11061, 17), (25807,), (11061,))

In [25]:
X_train.columns

Index(['board', 'market_segment', 'distribution_channel', 'room_type',
       'required_car_parking_spaces', 'special_requests', 'stay_nights',
       'rate', 'total_guests', 'hotel_type', 'country', 'parking',
       'total_rooms', 'restaurant', 'pool_and_spa', 'avg_review',
       'advance_reservation_days'],
      dtype='object')

In [26]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def process_data(X_train, X_test, numerical_cols, categorical_cols):
    """
    Realiza la normalización y la codificación one-hot de los datos.

    Args:
        X_train (pd.DataFrame): DataFrame de entrenamiento.
        X_test (pd.DataFrame): DataFrame de prueba.
        numerical_cols (list): Lista de columnas numéricas.
        categorical_cols (list): Lista de columnas categóricas.

    Returns:
        tuple: DataFrames de entrenamiento y prueba transformados.
    """

    # Pipelines para transformaciones numéricas y categóricas
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
    ])

    # ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ],
        remainder='passthrough'
    )

    # Aplicar transformaciones al conjunto de entrenamiento
    X_train_transformed = preprocessor.fit_transform(X_train)

    # Aplicar transformaciones al conjunto de prueba
    X_test_transformed = preprocessor.transform(X_test)

    # Obtener nombres de las columnas transformadas
    numerical_cols_transformed = [col for col in numerical_cols]
    categorical_cols_transformed = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_cols).tolist()
    remaining_cols = [col for col in X_train.columns if col not in numerical_cols + categorical_cols]

    transformed_columns = numerical_cols_transformed + categorical_cols_transformed + remaining_cols

    # Convertir a DataFrames de pandas
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=transformed_columns)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=transformed_columns)

    return X_train_transformed, X_test_transformed # Importa la función de procesamiento


# 3. Definir las columnas numéricas y categóricas para el procesamiento
numerical_cols = X_train.select_dtypes(include=['number']).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

# 4. Procesamiento (normalización y codificación one-hot)
X_train_processed, X_test_processed = process_data(X_train, X_test, numerical_cols, categorical_cols)

# Guardar los DataFrames procesados (opcional)
X_train_processed.to_csv('data/X_train_procesado.csv', index=False)
X_test_processed.to_csv('data/X_test_procesado.csv', index=False)


In [27]:
X_test_processed

,required_car_parking_spaces,special_requests,stay_nights,rate,total_guests,parking,total_rooms,restaurant,pool_and_spa,avg_review,...,room_type_E,room_type_F,room_type_G,room_type_H,room_type_L,hotel_type_City Hotel,hotel_type_Resort Hotel,country_FRA,country_POR,country_SPA
0,2.599861,1.511557,-0.333038,-0.406572,2.242741,1.422930,-0.446569,0.452869,1.135641,1.141449,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,-0.380400,-0.908416,-0.077437,1.061947,-1.715569,1.422930,0.569940,0.452869,1.135641,-0.532381,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,-0.380400,0.301571,-0.077437,0.103644,-0.396133,-0.702775,1.317373,0.452869,1.135641,-1.828249,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,-0.380400,-0.908416,0.178164,1.453551,-1.715569,1.422930,0.569940,0.452869,1.135641,-0.532381,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,-0.380400,0.301571,-0.333038,-0.252189,0.923304,-0.702775,-0.835234,-2.208146,-0.880560,-0.181416,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11056,-0.380400,1.511557,-0.588639,-1.573855,-0.396133,-0.702775,-1.194002,0.452869,-0.880560,-0.667367,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
11057,-0.380400,-0.908416,-0.077437,1.626761,-0.396133,-0.702775,1.407065,0.452869,-0.880560,1.384425,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
11058,-0.380400,0.301571,0.178164,1.137255,0.923304,-0.702775,-0.835234,-2.208146,-0.880560,-0.181416,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
11059,-0.380400,0.301571,-0.333038,-0.569740,0.923304,1.422930,0.569940,0.452869,1.135641,-0.532381,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [28]:
X_train_processed

,required_car_parking_spaces,special_requests,stay_nights,rate,total_guests,parking,total_rooms,restaurant,pool_and_spa,avg_review,...,room_type_E,room_type_F,room_type_G,room_type_H,room_type_L,hotel_type_City Hotel,hotel_type_Resort Hotel,country_FRA,country_POR,country_SPA
0,-0.380400,2.721544,-0.333038,-0.353856,-0.396133,-0.702775,1.407065,0.452869,-0.880560,1.384425,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2.599861,-0.908416,-0.588639,-0.902354,-0.396133,1.422930,-0.446569,0.452869,1.135641,1.141449,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,-0.380400,-0.908416,0.178164,0.111803,-1.715569,1.422930,0.569940,0.452869,1.135641,-0.532381,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,-0.380400,0.301571,0.178164,-0.318712,0.923304,-0.702775,-1.014618,0.452869,-0.880560,0.817482,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,-0.380400,-0.908416,-0.333038,-0.795666,-0.396133,-0.702775,1.407065,0.452869,-0.880560,1.384425,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25802,-0.380400,-0.908416,-0.333038,0.183346,-0.396133,-0.702775,1.407065,0.452869,-0.880560,1.384425,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
25803,-0.380400,-0.908416,-0.333038,-0.820769,-0.396133,-0.702775,1.407065,0.452869,-0.880560,1.384425,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
25804,-0.380400,1.511557,0.178164,2.266885,-0.396133,-0.702775,-1.194002,0.452869,-0.880560,-0.667367,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
25805,-0.380400,1.511557,0.433765,0.693248,-0.396133,-0.702775,1.407065,0.452869,-0.880560,1.384425,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
